In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import files
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# файлы импортируются из гугл диска
c = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/files/creditcard.csv')

In [9]:
# или с компьютера
c = pd.read_csv('creditcard.csv')

In [ ]:
c.head(10)

In [ ]:
c.info()

In [ ]:
print(c.isna().sum())

В датасете нет пропусков; один столбец содержит данные типа int (в данном случае они принимают значения 0 или 1), остальные столбцы содержат данные типа float